In [73]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# remove rows without invoice number and convert to string
df = pd.read_excel('Online Retail.xlsx')
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

# removing credit invoices / returns
df = df[~df['InvoiceNo'].str.contains('C')]

# filtering to United kingdom and grouping by invoice number and description, to show quantity of each item per order
market_basket = df[df['Country'] =="United Kingdom"].groupby(
                ['InvoiceNo', 'Description'])['Quantity']

products = df['Description'].unique()

# summing quantities of each item, item descriptions as columns and invoice number as row index
market_basket = market_basket.sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
market_basket.head()



Description,20713,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
def encode_data(datapoint):
    if datapoint <= 0:
        return 0
    if datapoint >= 1:
        return 1
    
market_basket = market_basket.applymap(encode_data)

In [61]:
itemsets = apriori(market_basket, min_support=0.01, use_colnames=True)


In [77]:
rules = association_rules(itemsets, metric="lift", min_threshold=1)
rules.count()
print(rules)

                        antecedents  \
0                  (DOTCOM POSTAGE)   
1          (6 RIBBONS RUSTIC CHARM)   
2          (JAM MAKING SET PRINTED)   
3          (6 RIBBONS RUSTIC CHARM)   
4          (6 RIBBONS RUSTIC CHARM)   
...                             ...   
3747    (CHARLOTTE BAG SUKI DESIGN)   
3748       (WOODLAND CHARLOTTE BAG)   
3749  (CHARLOTTE BAG PINK POLKADOT)   
3750     (STRAWBERRY CHARLOTTE BAG)   
3751  (RED RETROSPOT CHARLOTTE BAG)   

                                            consequents  antecedent support  \
0                              (6 RIBBONS RUSTIC CHARM)            0.037926   
1                                      (DOTCOM POSTAGE)            0.045800   
2                              (6 RIBBONS RUSTIC CHARM)            0.053996   
3                              (JAM MAKING SET PRINTED)            0.045800   
4                            (JAM MAKING SET WITH JARS)            0.045800   
...                                                 ...  

In [78]:
def get_consequents(input_item):
    consequents = []
    for i in range(len(rules)):
        for item in rules['antecedents'].iloc[i]:
            if item == input_item:
                for product in rules['consequents'].iloc[i]:
                    consequents.append(product)
    consequents = list(dict.fromkeys(consequents))
    return consequents

upsell_suggestions = get_consequents('JAM MAKING SET PRINTED')
print(upsell_suggestions)

['6 RIBBONS RUSTIC CHARM', 'DOTCOM POSTAGE', 'GREEN REGENCY TEACUP AND SAUCER', 'HEART OF WICKER SMALL', 'JAM MAKING SET WITH JARS', 'JUMBO BAG RED RETROSPOT', 'JUMBO SHOPPER VINTAGE RED PAISLEY', 'JUMBO STORAGE BAG SUKI', 'LUNCH BAG RED RETROSPOT', 'NATURAL SLATE HEART CHALKBOARD ', 'PACK OF 60 PINK PAISLEY CAKE CASES', 'PACK OF 72 RETROSPOT CAKE CASES', 'RECIPE BOX PANTRY YELLOW DESIGN', 'RECYCLING BAG RETROSPOT ', 'RED TOADSTOOL LED NIGHT LIGHT', 'REGENCY CAKESTAND 3 TIER', 'ROSES REGENCY TEACUP AND SAUCER ', 'SET OF 3 CAKE TINS PANTRY DESIGN ', 'SET OF 4 PANTRY JELLY MOULDS', 'SET OF 6 SPICE TINS PANTRY DESIGN', 'SUKI  SHOULDER BAG', 'VINTAGE SNAP CARDS', 'WHITE HANGING HEART T-LIGHT HOLDER']


In [75]:
products

array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ..., 'lost',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], dtype=object)

In [79]:
recommendations = []

for p in products:
    upsell_suggestions = get_consequents(p)
    recommendations.append({
        'Product': p,
        'Recommendations': upsell_suggestions
    })
    
recommendations = pd.DataFrame(recommendations)
recommendations

,Product,Recommendations
0,WHITE HANGING HEART T-LIGHT HOLDER,"[6 RIBBONS RUSTIC CHARM, 60 TEATIME FAIRY CAKE..."
1,WHITE METAL LANTERN,[]
2,CREAM CUPID HEARTS COAT HANGER,[]
3,KNITTED UNION FLAG HOT WATER BOTTLE,[]
4,RED WOOLLY HOTTIE WHITE HEART.,[]
...,...,...
4203,????damages????,[]
4204,mixed up,[]
4205,lost,[]
4206,CREAM HANGING HEART T-LIGHT HOLDER,[]


In [80]:
recommendations.head(20)

,Product,Recommendations
0,WHITE HANGING HEART T-LIGHT HOLDER,"[6 RIBBONS RUSTIC CHARM, 60 TEATIME FAIRY CAKE..."
1,WHITE METAL LANTERN,[]
2,CREAM CUPID HEARTS COAT HANGER,[]
3,KNITTED UNION FLAG HOT WATER BOTTLE,[]
4,RED WOOLLY HOTTIE WHITE HEART.,[]
5,SET 7 BABUSHKA NESTING BOXES,[]
6,GLASS STAR FROSTED T-LIGHT HOLDER,[]
7,HAND WARMER UNION JACK,"[HAND WARMER OWL DESIGN, HAND WARMER RED LOVE ..."
8,HAND WARMER RED POLKA DOT,[]
9,ASSORTED COLOUR BIRD ORNAMENT,"[PAINTED METAL PEARS ASSORTED, PARTY BUNTING, ..."
